In [26]:
import pandas as pd
import pickle as pk
import plotly.express as px

In [27]:
with open('../src/depara/trends/categoria.pkl', 'rb') as arq:
    df_categorias = pk.load(arq)
    df_categorias['ID'] = df_categorias['ID'].astype('int32')
    df_categorias['NOME_CATEGORIA'] = df_categorias['NOME_CATEGORIA'].astype('string')
df_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              31 non-null     int32 
 1   NOME_CATEGORIA  31 non-null     string
dtypes: int32(1), string(1)
memory usage: 500.0 bytes


In [31]:
opcoes_categoria = []
for chave, linha in df_categorias.iterrows():
    dic_opcoes = {
        'label': linha['NOME_CATEGORIA'],
        'value': linha['ID']
    }
    opcoes_categoria.append(dic_opcoes)
opcoes_categoria

[{'label': 'Film & Animation', 'value': 1},
 {'label': 'Autos & Vehicles', 'value': 2},
 {'label': 'Music', 'value': 10},
 {'label': 'Pets & Animals', 'value': 15},
 {'label': 'Sports', 'value': 17},
 {'label': 'Short Movies', 'value': 18},
 {'label': 'Travel & Events', 'value': 19},
 {'label': 'Gaming', 'value': 20},
 {'label': 'Videoblogging', 'value': 21},
 {'label': 'People & Blogs', 'value': 22},
 {'label': 'Comedy', 'value': 23},
 {'label': 'Entertainment', 'value': 24},
 {'label': 'News & Politics', 'value': 25},
 {'label': 'Howto & Style', 'value': 26},
 {'label': 'Education', 'value': 27},
 {'label': 'Science & Technology', 'value': 28},
 {'label': 'Movies', 'value': 30},
 {'label': 'Anime/Animation', 'value': 31},
 {'label': 'Action/Adventure', 'value': 32},
 {'label': 'Classics', 'value': 33},
 {'label': 'Comedy', 'value': 34},
 {'label': 'Documentary', 'value': 35},
 {'label': 'Drama', 'value': 36},
 {'label': 'Family', 'value': 37},
 {'label': 'Foreign', 'value': 38},
 {'l

In [12]:
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet')

In [13]:
base_trends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_extracao          4400 non-null   string 
 1   DATA_PUBLICACAO        4400 non-null   string 
 2   ID_CANAL               4400 non-null   string 
 3   NM_CANAL               4400 non-null   string 
 4   ID_CATEGORIA           4400 non-null   int32  
 5   ID_VIDEO               4400 non-null   string 
 6   TITULO_VIDEO           4400 non-null   string 
 7   DESCRICAO              4400 non-null   string 
 8   TAGS                   3661 non-null   string 
 9   DURACAO_VIDEOS         4400 non-null   object 
 10  TOTAL_VISUALIZACOES    4400 non-null   int32  
 11  TOTAL_LIKES            4400 non-null   int32  
 12  TOTAL_FAVORITOS        4400 non-null   int32  
 13  TOTAL_COMENTARIOS      4400 non-null   int32  
 14  data_hora_extracao     4400 non-null   string 
 15  DURA

# Métricas Trends

- Categorias Populares por data de extração top 10

In [14]:
#  'TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_COMENTARIOS'
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'ID_VIDEO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' data_extracao == "2024-01-18" and INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='ID_VIDEO')

,data_extracao,ID_CATEGORIA,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_VIDEO,TOTAL_VISUALIZACOES
2507,2024-01-18,24,Noite,3,-7a68_XfpG4,679320
1551,2024-01-18,17,Noite,3,-A7X9LMN0qU,114810
41,2024-01-18,17,Noite,3,-JzAKMfuWy4,223287
733,2024-01-18,24,Noite,3,-U5oLyhfq0A,172139
3672,2024-01-18,24,Noite,3,-cGZ4mFlAFY,325350
...,...,...,...,...,...,...
1096,2024-01-18,20,Noite,3,yJzvSZ7EKN8,1451661
987,2024-01-18,20,Noite,3,yN1u45QKPVw,152720
4077,2024-01-18,17,Noite,3,yQPIQ97nYG4,278899
2590,2024-01-18,28,Noite,3,zKmNviriS50,65072


In [15]:
base_trends.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 41 to 4357
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   data_extracao          200 non-null    string
 1   ID_CATEGORIA           200 non-null    int32 
 2   TURNO_EXTRACAO         200 non-null    string
 3   INDICE_TURNO_EXTRACAO  200 non-null    string
 4   ID_VIDEO               200 non-null    string
 5   TOTAL_VISUALIZACOES    200 non-null    int32 
dtypes: int32(2), string(4)
memory usage: 9.4 KB


In [16]:
base_trends = base_trends.groupby(['data_extracao', 'ID_CATEGORIA']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')

    ).reset_index()

base_trends

,data_extracao,ID_CATEGORIA,TOTAL_MAX
0,2024-01-18,1,823820
1,2024-01-18,2,161708
2,2024-01-18,10,81801902
3,2024-01-18,17,30452063
4,2024-01-18,19,1074972
5,2024-01-18,20,124769848
6,2024-01-18,22,12286382
7,2024-01-18,23,2277480
8,2024-01-18,24,125759485
9,2024-01-18,25,5939219


In [17]:
base_trends_completa = pd.merge(
    right=base_trends,
    left=df_categorias,
    how='inner',
    left_on='ID',
    right_on='ID_CATEGORIA'
).sort_values(by='TOTAL_MAX', ascending=False)
base_trends_completa

,ID,NOME_CATEGORIA,data_extracao,ID_CATEGORIA,TOTAL_MAX
8,24,Entertainment,2024-01-18,24,125759485
5,20,Gaming,2024-01-18,20,124769848
2,10,Music,2024-01-18,10,81801902
3,17,Sports,2024-01-18,17,30452063
6,22,People & Blogs,2024-01-18,22,12286382
11,28,Science & Technology,2024-01-18,28,7185043
9,25,News & Politics,2024-01-18,25,5939219
7,23,Comedy,2024-01-18,23,2277480
4,19,Travel & Events,2024-01-18,19,1074972
0,1,Film & Animation,2024-01-18,1,823820


In [25]:
fig = px.bar(
    base_trends_completa,
    x='TOTAL_MAX',
    y='NOME_CATEGORIA',
    orientation='h',
    category_orders={'NOME_CATEGORIA': base_trends_completa['NOME_CATEGORIA'].tolist()}

)
fig.show()

In [117]:

base_trends_completa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 8 to 1
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              12 non-null     int32 
 1   NOME_CATEGORIA  12 non-null     string
 2   data_extracao   12 non-null     string
 3   ID_CATEGORIA    12 non-null     int32 
 4   TOTAL_MAX       12 non-null     int32 
dtypes: int32(3), string(2)
memory usage: 432.0 bytes


- Canais populares (likes, comentarios e visualizações) Por categoria TOP 10

In [118]:
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' ID_CATEGORIA == 20 and data_extracao == "2024-01-19"  and  INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='ID_CANAL')

,data_extracao,ID_CATEGORIA,ID_CANAL,NM_CANAL,ID_VIDEO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES
1727,2024-01-19,20,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,ZHhqwBwmRkI,Noite,3,104889364
2459,2024-01-19,20,UC6P6SvP2arhEdJ9Pg8qwP2A,Athos,RVoUKeKn7q0,Noite,3,705586
2777,2024-01-19,20,UC8rAKIddyMROk00d6Kebi0g,KillerProductionsful,34ei8y1vjOU,Noite,3,557654
2110,2024-01-19,20,UC9Qu6o04oTf-b2acahQVk1g,Kass e KR,WZMFR2HHPto,Noite,3,857416
920,2024-01-19,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,RApm1yPoE-0,Noite,3,157753
935,2024-01-19,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,g7bRCHLCs1g,Noite,3,156645
2423,2024-01-19,20,UCBiRuk3NRX_SxCsp3l3cLAw,Xou,wq0TJjlggrM,Noite,3,72519
2202,2024-01-19,20,UCEfGV5hx2VrXl4jOUnG0MRQ,Geleia,xmnrb2Q6D44,Noite,3,803370
806,2024-01-19,20,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,yJzvSZ7EKN8,Noite,3,1676719
1308,2024-01-19,20,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,kVzwBImBoGU,Noite,3,1325611


In [107]:
base_trends_canal_top_dez = base_trends.groupby(['data_extracao', 'ID_CANAL', 'NM_CANAL']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')
    ).reset_index()

base_trends_canal_top_dez.nlargest(10, columns=['TOTAL_MAX'])

,data_extracao,ID_CANAL,NM_CANAL,TOTAL_MAX
0,2024-01-19,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,104889364
7,2024-01-19,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,4517914
29,2024-01-19,UCrcBZDofBltcgGxphNia2tw,Propolis,2975865
17,2024-01-19,UCWsDFcIhY2DBi3GB5uykGXA,IShowSpeed,1295946
15,2024-01-19,UCVse66mti9GN2SWF2eh_SCg,FREITAS,1226690
23,2024-01-19,UCiS882YPwZt1NfaM0gR0D9Q,Genshin Impact,1177554
24,2024-01-19,UCjBp_7RuDBUYbd1LegWEJ8g,Xbox,1071090
3,2024-01-19,UC9Qu6o04oTf-b2acahQVk1g,Kass e KR,857416
6,2024-01-19,UCEfGV5hx2VrXl4jOUnG0MRQ,Geleia,803370
21,2024-01-19,UCeoaBlUFa1wvkXnkYgsAY3A,GonGon,708432


In [119]:

base_trends_canal_top_dez.nsmallest(10, columns=['TOTAL_MAX'])

,data_extracao,ID_CANAL,NM_CANAL,TOTAL_MAX
5,2024-01-19,UCBiRuk3NRX_SxCsp3l3cLAw,Xou,72519
30,2024-01-19,UCsTNNzGmdiiUfRJqDpl1GKw,Laboratório de Zaun,104614
10,2024-01-19,UCKxn56z2qU-LJi3KRgMiaUg,Casimiro,153907
27,2024-01-19,UCot9DELv0Avr0HclcekQCpg,MODO CARREIRA SOTO,193594
20,2024-01-19,UCa2z1-KuJYbJ5cKHD_QUDoQ,Açúcar,208320
11,2024-01-19,UCLjzb0-xUi_gfnZGd9ZgRwQ,Richard GameVlogs,223052
9,2024-01-19,UCK3NkZPs0BurxabhtnSDa3g,Souzones,224081
16,2024-01-19,UCWKtHaeXVzUscYGcm0hEunw,BRKsEDU,228700
12,2024-01-19,UCQEUV7le_zYKNKGmUJOwvBw,Bauers,256909
14,2024-01-19,UCUouM5ZRqZoFNzUvpDeYNGQ,Lucas Clash ON,277186


- Desempenho da categoria por dia

In [75]:
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' ID_CATEGORIA == 20 and  INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='data_extracao')

,data_extracao,ID_CATEGORIA,ID_CANAL,NM_CANAL,ID_VIDEO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES
2887,2024-01-17,20,UCVse66mti9GN2SWF2eh_SCg,FREITAS,xNpIg_GyxrA,Noite,3,521975
2588,2024-01-17,20,UCH2958aySlHIuMzpUE8Xe9A,Tex HS,YTLiUbSMK1o,Noite,3,651193
1215,2024-01-17,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,RApm1yPoE-0,Noite,3,138432
1211,2024-01-17,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,RApm1yPoE-0,Noite,3,138621
1192,2024-01-17,20,UC6P6SvP2arhEdJ9Pg8qwP2A,Athos,gy_sxyoMwhE,Noite,3,1397335
...,...,...,...,...,...,...,...,...
2810,2024-01-23,20,UCvp2rct896qzl9qaG20mjNg,GamerCosta,oYiey5uDcwI,Noite,3,545693
1990,2024-01-23,20,UCrcBZDofBltcgGxphNia2tw,Propolis,skMrlmbvqdg,Noite,3,915612
3170,2024-01-23,20,UCUouM5ZRqZoFNzUvpDeYNGQ,Lucas Clash ON,UdXbmiZdBX4,Noite,3,401292
1705,2024-01-23,20,UCiS882YPwZt1NfaM0gR0D9Q,Genshin Impact,zQk1QHm2-Cw,Noite,3,1059536


In [79]:
base_trends = base_trends.groupby(['data_extracao']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')
    ).reset_index().sort_values(by='data_extracao')

In [80]:
base_trends

,data_extracao,TOTAL_MAX
0,2024-01-17,248129261
1,2024-01-18,124769848
2,2024-01-19,127554063
3,2024-01-20,130299289
4,2024-01-21,130388361
5,2024-01-22,23721598
6,2024-01-23,23290214


In [81]:
base_trends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   data_extracao  7 non-null      string
 1   TOTAL_MAX      7 non-null      int32 
dtypes: int32(1), string(1)
memory usage: 212.0 bytes


- Top 10 vídeo em alta por categoria  selecionando dia

In [94]:
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'ID_VIDEO', 'TITULO_VIDEO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' data_extracao == "2024-01-18" and INDICE_TURNO_EXTRACAO == "3" and ID_CATEGORIA == 20')
base_trends.sort_values(by='ID_VIDEO')

,data_extracao,ID_CATEGORIA,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES
1754,2024-01-18,20,Noite,3,0KlQMBnOK2w,É TUDO QUE A GENTE QUERIA... REACT ANIMAÇÃO TE...,104035
98,2024-01-18,20,Noite,3,2wpznW4qAu0,SOBREVIVI COMO SERPENTE GODZILLA DINOSSAURO 10...,217115
2911,2024-01-18,20,Noite,3,34ei8y1vjOU,O ALDEÃO FAVORITO DO GROX! | MINECRAFT | GroxMC,511855
1791,2024-01-18,20,Noite,3,57ywwPKMEbA,ZEREI DEEEER SIMULATOR SÓ PRA VER 9 JAZZGHOST ...,1017234
1698,2024-01-18,20,Noite,3,A0u8rgz4qlA,Life In Brazil🇧🇷,1066087
1636,2024-01-18,20,Noite,3,CPWeiZKO-Vw,Revelando os 10 Piores Segredos do PROPOLIS,1104138
3599,2024-01-18,20,Noite,3,FAiNpxAadI4,O PRÓPOLIS ̷I̷N̷V̷A̷D̷I̷U̷ MEU CANAL...,334302
2642,2024-01-18,20,Noite,3,GqQEKL7o7Cc,A VOLTA DA GALINHA ATACADA! - Chicken Feet Bre...,627745
4034,2024-01-18,20,Noite,3,O4jdx0CzVZg,EU TRANSFORMEI MINHA BASE NO MINECRAFT,283285
83,2024-01-18,20,Noite,3,O7BueZ5WyLs,RETALIAÇÃO // Animação do Episódio 8 – VALORANT,218942


In [96]:
base_trends_top_dez =  base_trends.groupby(['data_extracao', 'ID_VIDEO']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')
    ).reset_index().sort_values(by='TOTAL_MAX', ascending=False)
base_trends_top_dez.nlargest(10, columns=['TOTAL_MAX'])

,data_extracao,ID_VIDEO,TOTAL_MAX
20,2024-01-18,ZHhqwBwmRkI,104466364
40,2024-01-18,yJzvSZ7EKN8,1451661
30,2024-01-18,kVzwBImBoGU,1270814
13,2024-01-18,SsXVHL69nd0,1238618
5,2024-01-18,CPWeiZKO-Vw,1104138
4,2024-01-18,A0u8rgz4qlA,1066087
3,2024-01-18,57ywwPKMEbA,1017234
35,2024-01-18,skMrlmbvqdg,854675
16,2024-01-18,WZMFR2HHPto,848176
32,2024-01-18,o_MQBA2shDk,845421


In [108]:
base_trends_top_dez.nsmallest(10, columns=['TOTAL_MAX'])

,data_extracao,ID_VIDEO,TOTAL_MAX
0,2024-01-18,0KlQMBnOK2w,104035
10,2024-01-18,RApm1yPoE-0,151055
17,2024-01-18,YI_1rXssqzU,152161
41,2024-01-18,yN1u45QKPVw,152720
22,2024-01-18,aBugQ1P5MHg,153978
27,2024-01-18,hqmR7qhBXOk,179434
26,2024-01-18,gjoxK9x9ljM,184677
34,2024-01-18,qcoVg6RWko8,190492
12,2024-01-18,RYKkM4qYLVo,193198
37,2024-01-18,wyCIS_526WE,205850


- Engajamento por canal Top 10 selecionando categoria e data

In [19]:

colunas = ['data_extracao', 'ID_CATEGORIA' , 'INDICE_TURNO_EXTRACAO', 'ID_CANAL', 'NM_CANAL', 'TITULO_VIDEO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_COMENTARIOS']
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' data_extracao == "2024-01-18" and INDICE_TURNO_EXTRACAO == "3" and ID_CATEGORIA == 20')
base_trends.head(5)

,data_extracao,ID_CATEGORIA,INDICE_TURNO_EXTRACAO,ID_CANAL,NM_CANAL,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_LIKES,TOTAL_COMENTARIOS
65,2024-01-18,20,3,UCK3NkZPs0BurxabhtnSDa3g,Souzones,GOOSE GOOSE DUCK: MINHA REDENÇÃO APÓS A PIOR P...,221033,29822,927
83,2024-01-18,20,3,UCgWiuB2PQIUhJgEN9No281g,VALORANT // Brasil,RETALIAÇÃO // Animação do Episódio 8 – VALORANT,218942,25402,898
98,2024-01-18,20,3,UCLjzb0-xUi_gfnZGd9ZgRwQ,Richard GameVlogs,SOBREVIVI COMO SERPENTE GODZILLA DINOSSAURO 10...,217115,5458,80
205,2024-01-18,20,3,UCWKtHaeXVzUscYGcm0hEunw,BRKsEDU,THE LAST OF US PART II REMASTERED - Gameplay d...,207627,17813,764
230,2024-01-18,20,3,UCa2z1-KuJYbJ5cKHD_QUDoQ,Açúcar,FIM DO CANAL... Propolis INVADIU meu canal,205850,8014,1268


In [22]:
base_trends.groupby(['ID_CANAL', 'NM_CANAL']) \
    .agg(
        TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'sum'),
        TOTAL_LIKES=('TOTAL_LIKES', 'sum'),
        TOTAL_COMENTARIOS=('TOTAL_VISUALIZACOES', 'sum'),
    ).reset_index()

,ID_CANAL,NM_CANAL,TOTAL_VISUALIZACOES,TOTAL_LIKES,TOTAL_COMENTARIOS
0,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,104466364,1760712,104466364
1,UC6P6SvP2arhEdJ9Pg8qwP2A,Athos,661846,13180,661846
2,UC8rAKIddyMROk00d6Kebi0g,KillerProductionsful,511855,40956,511855
3,UC9Qu6o04oTf-b2acahQVk1g,Kass e KR,848176,45368,848176
4,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,341547,46853,341547
5,UCEfGV5hx2VrXl4jOUnG0MRQ,Geleia,740488,25120,740488
6,UCH2958aySlHIuMzpUE8Xe9A,Tex HS,667992,30495,667992
7,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,3739709,206739,3739709
8,UCJIl6Ahgkfwg1DRYcg-kY3g,Mendrake,385537,18691,385537
9,UCK3NkZPs0BurxabhtnSDa3g,Souzones,221033,29822,221033


In [23]:
base_trends['TAXA_ENGAJAMENTO'] = round(((base_trends['TOTAL_LIKES'] + base_trends['TOTAL_COMENTARIOS']) / base_trends['TOTAL_VISUALIZACOES']) * 100, 2)
base_trends_top_dez = base_trends.nlargest(10, columns=['TAXA_ENGAJAMENTO'])
base_trends_top_dez[['data_extracao', 'ID_CANAL', 'NM_CANAL','TOTAL_VISUALIZACOES' ,  'TOTAL_COMENTARIOS', 'TOTAL_LIKES', 'TAXA_ENGAJAMENTO']]

,data_extracao,ID_CANAL,NM_CANAL,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES,TAXA_ENGAJAMENTO
2642,2024-01-18,UCYzp8-P162Em-3uZnCuGJOg,Games EduUu,627745,4598,108450,18.01
453,2024-01-18,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,190492,1243,29357,16.06
65,2024-01-18,UCK3NkZPs0BurxabhtnSDa3g,Souzones,221033,927,29822,13.91
1012,2024-01-18,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,151055,721,17496,12.06
83,2024-01-18,UCgWiuB2PQIUhJgEN9No281g,VALORANT // Brasil,218942,898,25402,12.01
996,2024-01-18,UCfSCj6pb_AMSszt73mqgP_Q,Lggj,152161,706,17062,11.68
398,2024-01-18,UCot9DELv0Avr0HclcekQCpg,MODO CARREIRA SOTO,193198,967,21180,11.46
1754,2024-01-18,UCsTNNzGmdiiUfRJqDpl1GKw,Laboratório de Zaun,104035,1128,10646,11.32
624,2024-01-18,UCfSCj6pb_AMSszt73mqgP_Q,Lggj,179434,1439,18316,11.01
550,2024-01-18,UCfSCj6pb_AMSszt73mqgP_Q,Lggj,184677,1149,17401,10.04


- Engajamento do vídeo por categoria

In [24]:
colunas = ['data_extracao', 'ID_CATEGORIA' , 'INDICE_TURNO_EXTRACAO', 'NM_CANAL', 'ID_VIDEO',  'TITULO_VIDEO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_COMENTARIOS']
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' data_extracao == "2024-01-18" and INDICE_TURNO_EXTRACAO == "3" and ID_CATEGORIA == 20')
base_trends.head(5)

,data_extracao,ID_CATEGORIA,INDICE_TURNO_EXTRACAO,NM_CANAL,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_LIKES,TOTAL_COMENTARIOS
65,2024-01-18,20,3,Souzones,jn6kT6YBdOY,GOOSE GOOSE DUCK: MINHA REDENÇÃO APÓS A PIOR P...,221033,29822,927
83,2024-01-18,20,3,VALORANT // Brasil,O7BueZ5WyLs,RETALIAÇÃO // Animação do Episódio 8 – VALORANT,218942,25402,898
98,2024-01-18,20,3,Richard GameVlogs,2wpznW4qAu0,SOBREVIVI COMO SERPENTE GODZILLA DINOSSAURO 10...,217115,5458,80
205,2024-01-18,20,3,BRKsEDU,cMU4YQ3ZrbM,THE LAST OF US PART II REMASTERED - Gameplay d...,207627,17813,764
230,2024-01-18,20,3,Açúcar,wyCIS_526WE,FIM DO CANAL... Propolis INVADIU meu canal,205850,8014,1268


In [29]:
base_trends['TAXA_ENGAJAMENTO'] = round(((base_trends['TOTAL_LIKES'] + base_trends['TOTAL_COMENTARIOS']) / base_trends['TOTAL_VISUALIZACOES']) * 100, 2)
base_trends_top_dez = base_trends.nlargest(10, columns=['TAXA_ENGAJAMENTO'])
base_trends_top_dez[['data_extracao',  'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO','TOTAL_VISUALIZACOES' ,  'TOTAL_COMENTARIOS', 'TOTAL_LIKES', 'TAXA_ENGAJAMENTO']]

,data_extracao,NM_CANAL,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES,TAXA_ENGAJAMENTO
2642,2024-01-18,Games EduUu,GqQEKL7o7Cc,A VOLTA DA GALINHA ATACADA! - Chicken Feet Bre...,627745,4598,108450,18.01
453,2024-01-18,Apuh,qcoVg6RWko8,ESSE É O FIM DO DIA DO EXPURGO... - Irmandade,190492,1243,29357,16.06
65,2024-01-18,Souzones,jn6kT6YBdOY,GOOSE GOOSE DUCK: MINHA REDENÇÃO APÓS A PIOR P...,221033,927,29822,13.91
1012,2024-01-18,Apuh,RApm1yPoE-0,FINALMENTE TIVE MINHA VINGANÇA!! DIA DO EXPURG...,151055,721,17496,12.06
83,2024-01-18,VALORANT // Brasil,O7BueZ5WyLs,RETALIAÇÃO // Animação do Episódio 8 – VALORANT,218942,898,25402,12.01
996,2024-01-18,Lggj,YI_1rXssqzU,FIZ UM BUNKER À PROVA DE DIA DO EXPURGO - Irm...,152161,706,17062,11.68
398,2024-01-18,MODO CARREIRA SOTO,RYKkM4qYLVo,Dorival promete ACABAR com a MOLEZA de MUITOS ...,193198,967,21180,11.46
1754,2024-01-18,Laboratório de Zaun,0KlQMBnOK2w,É TUDO QUE A GENTE QUERIA... REACT ANIMAÇÃO TE...,104035,1128,10646,11.32
624,2024-01-18,Lggj,hqmR7qhBXOk,A GRANDE VINGANÇA - Irmandade,179434,1439,18316,11.01
550,2024-01-18,Lggj,gjoxK9x9ljM,"A SEMANA DO EXPURGO, 168 HORAS SEM REGRAS NESS...",184677,1149,17401,10.04
